**Cloud HPC Instance Ranking using Multi-Criteria Decision Analysis (MCDA)**

This JupyterLab notebook implements a multi-criteria decision analysis (MCDA) framework for evaluating and ranking cloud HPC instances using the Technique for Order Preference by Similarity to Ideal Solution (TOPSIS) method. The notebook integrates various statistical techniques, including sensitivity analysis, bootstrapping, and the Friedman test, to ensure the robustness and reliability of the rankings.

The ranking process involves evaluating cloud HPC instances from major cloud platforms (AWS, Google Cloud Platform, Microsoft Azure, and Oracle Cloud Infrastructure) based on key parameters: physical CPU cores, total memory, memory per core, network bandwidth, and on-demand hourly cost. The notebook offers a systematic approach to selecting the most suitable cloud HPC instance for high-performance workloads, considering both performance and cost factors.

This JupyterLab implementation enables users to replicate the methodology, perform sensitivity analysis to assess the stability of rankings, apply bootstrapping for uncertainty quantification, and validate results using the Friedman test for statistical significance. It provides an effective tool for decision-making in cloud HPC instance selection.

Import necessary Python libraries

In [35]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata, friedmanchisquare
import seaborn as sns

# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [36]:
# Define the TOPSIS function with epsilon to avoid division by zero
def topsis(raw_data, weights, benefit_categories, epsilon=1e-10):
    m, n = raw_data.shape
    # Normalize the raw data
    divisors = np.sqrt(np.sum(raw_data ** 2, axis=0))
    normalized_data = raw_data / divisors

    # Apply weights
    weighted_data = normalized_data * weights

    # Determine Ideal and Negative Ideal Solutions
    ideal_solution = np.zeros(n)
    negative_ideal_solution = np.zeros(n)
    for j in range(n):
        if j in benefit_categories:
            ideal_solution[j] = np.max(weighted_data[:, j])
            negative_ideal_solution[j] = np.min(weighted_data[:, j])
        else:
            ideal_solution[j] = np.min(weighted_data[:, j])
            negative_ideal_solution[j] = np.max(weighted_data[:, j])

    # Calculate distances
    dist_to_ideal = np.sqrt(np.sum((weighted_data - ideal_solution) ** 2, axis=1))
    dist_to_negative_ideal = np.sqrt(np.sum((weighted_data - negative_ideal_solution) ** 2, axis=1))

    # Calculate TOPSIS scores with epsilon to prevent division by zero
    scores = dist_to_negative_ideal / (dist_to_ideal + dist_to_negative_ideal + epsilon)
    return scores

**Identification of Criteria and Weights**

Define the decision matrix, criteria, and weights for the alternatives.

In [37]:
# Identification of Criteria and Weights
categories = np.array(["Physical CPU Cores (PC)", "Total Memory (TM)", "Memory/Core (MC)", "Network Bandwidth (NB)", "On-Demand Hourly Cost (HC)"])
alternatives = np.array(["AWS hpc7g.4xlarge", "AWS hpc7g.8xlarge", "AWS hpc7g.16xlarge", "AWS hpc7a.12xlarge", "AWS hpc7a.24xlarge", "AWS hpc7a.48xlarge", "AWS hpc7a.96xlarge", "AWS hpc6id.32xlarge", "AWS hpc6a.48xlarge", "Azure Standard_HB60-15rs", "Azure Standard_HB60-30rs", "Azure Standard_HB60-45rs", "Azure Standard_HB60rs", "Azure Standard_HB120-16rs_v2", "Azure Standard_HB120-32rs_v2", "Azure Standard_HB120-64rs_v2", "Azure Standard_HB120-96rs_v2", "Azure Standard_HB120rs_v2", "Azure Standard_HB120-16rs_v3", "Azure Standard_HB120-32rs_v3", "Azure Standard_HB120-64rs_v3", "Azure Standard_HB120-96rs_v3", "Azure Standard_HB120rs_v3", "Azure Standard_HB176-24rs_v4", "Azure Standard_HB176-48rs_v4", "Azure Standard_HB176-96rs_v4", "Azure Standard_HB176-144rs_v4", "Azure Standard_HB176rs_v4", "Azure Standard_HC44-16rs", "Azure Standard_HC44-32rs", "Azure Standard_HC44rs", "Azure Standard_HX176-24rs", "Azure Standard_HX176-48rs", "Azure Standard_HX176-96rs", "Azure Standard_HX176-144rs", "Azure Standard_HX176rs", "GCP h3-standard-88", "OCI BM.HPC.E5.144", "OCI BM.Optimized3.36", "OCI BM.HPC2.36"])
raw_data = np.array([
[16, 128,	8, 200, 1.6832],
[32, 128, 4, 200, 1.6832],
[64, 128, 2, 200, 1.6832],
[24, 768, 32, 300, 7.2],
[48, 768,	16, 300, 7.2],
[96, 768, 8, 300, 7.2],
[192, 768, 4, 300, 7.2],
[64, 1024, 16, 200, 5.7],
[96, 384, 4, 100, 2.88],
[15, 228, 15.2, 100, 2.28],
[30, 228, 7.6, 100, 2.28],
[45, 228, 5.067, 100, 2.28],
[60, 228, 3.8, 100, 2.28],
[16, 456, 28.5, 200, 3.6],
[32, 456, 14.25, 200, 3.6],
[64, 456,	7.125, 200, 3.6],
[96, 456, 4.75, 200, 3.6],
[120, 456, 3.8, 200, 3.6],
[16, 456, 28.5, 200, 3.6],
[32, 456, 14.25, 200, 3.6],
[64, 456, 7.125, 200, 3.6],
[96, 456, 4.75, 200, 3.6],
[120, 456, 3.8, 200, 3.6],
[24, 768, 32, 400, 7.2],
[48, 768, 16, 400, 7.2],
[96, 768,	8, 400, 7.2],
[144, 768, 5.33, 400, 7.2],
[176, 768, 4.36, 400, 7.2],
[16, 352, 22, 100, 3.168],
[32, 352, 11, 100, 3.168],
[44, 352, 8, 100, 3.168],
[24, 1408, 58.67, 400, 8.64],
[48, 1408, 29.33, 400, 8.64],
[96, 1408, 14.67, 400, 8.64],
[144, 1408, 9.78, 400, 8.64],
[176, 1408, 8, 400, 8.64],
[88, 352, 4, 200, 4.9236],
[144, 768, 5.33, 100, 6.34],
[36, 512, 14.22, 100, 2.71],
[36, 384, 10.67, 100, 2.7],
])

initial_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
benefit_categories = set([0, 1, 2, 3])

# Display raw data and weights
raw_data_df = pd.DataFrame(data=raw_data, index=alternatives, columns=categories)
weights_df = pd.DataFrame(data=initial_weights, index=categories, columns=["Weights"])

print("Raw Data:")
display(raw_data_df)
print("Initial Weights:")
display(weights_df)

Raw Data:


,Physical CPU Cores (PC),Total Memory (TM),Memory/Core (MC),Network Bandwidth (NB),On-Demand Hourly Cost (HC)
AWS hpc7g.4xlarge,16.0,128.0,8.000,200.0,1.6832
AWS hpc7g.8xlarge,32.0,128.0,4.000,200.0,1.6832
AWS hpc7g.16xlarge,64.0,128.0,2.000,200.0,1.6832
AWS hpc7a.12xlarge,24.0,768.0,32.000,300.0,7.2000
AWS hpc7a.24xlarge,48.0,768.0,16.000,300.0,7.2000
AWS hpc7a.48xlarge,96.0,768.0,8.000,300.0,7.2000
AWS hpc7a.96xlarge,192.0,768.0,4.000,300.0,7.2000
AWS hpc6id.32xlarge,64.0,1024.0,16.000,200.0,5.7000
AWS hpc6a.48xlarge,96.0,384.0,4.000,100.0,2.8800
Azure Standard_HB60-15rs,15.0,228.0,15.200,100.0,2.2800


Initial Weights:


,Weights
Physical CPU Cores (PC),0.2
Total Memory (TM),0.2
Memory/Core (MC),0.2
Network Bandwidth (NB),0.2
On-Demand Hourly Cost (HC),0.2


**Normalization of Data**

Normalization is essential to bring all criteria to a common scale, ensuring that each criterion contributes proportionally to the decision-making process. This step involves transforming the raw data for each criterion into a dimensionless value between 0 and 1. Various normalization techniques, such as min-max normalization or z-score normalization, can be applied depending on the nature of the data.


In [38]:
# Normalize the raw data
m, n = raw_data.shape
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:, j]
    divisors[j] = np.sqrt(column @ column)
normalized_data = raw_data / divisors

# Normalize the weights to ensure that they sum up to 1
weights = initial_weights / np.sum(initial_weights)

normalized_data_df = pd.DataFrame(data=normalized_data, index=alternatives, columns=categories)

print("Normalized Data:")
display(normalized_data_df)

Normalized Data:


,Physical CPU Cores (PC),Total Memory (TM),Memory/Core (MC),Network Bandwidth (NB),On-Demand Hourly Cost (HC)
AWS hpc7g.4xlarge,0.029494,0.028446,0.075094,0.122398,0.049664
AWS hpc7g.8xlarge,0.058988,0.028446,0.037547,0.122398,0.049664
AWS hpc7g.16xlarge,0.117976,0.028446,0.018774,0.122398,0.049664
AWS hpc7a.12xlarge,0.044241,0.170674,0.300377,0.183597,0.212440
AWS hpc7a.24xlarge,0.088482,0.170674,0.150188,0.183597,0.212440
AWS hpc7a.48xlarge,0.176965,0.170674,0.075094,0.183597,0.212440
AWS hpc7a.96xlarge,0.353929,0.170674,0.037547,0.183597,0.212440
AWS hpc6id.32xlarge,0.117976,0.227565,0.150188,0.122398,0.168181
AWS hpc6a.48xlarge,0.176965,0.085337,0.037547,0.061199,0.084976
Azure Standard_HB60-15rs,0.027651,0.050669,0.142679,0.061199,0.067273


The weights are normalized to ensure that they sum up to 1.

In [39]:
# Weighted normalized decision matrix
weighted_data = normalized_data * weights

weighted_data_df = pd.DataFrame(data=weighted_data, index=alternatives, columns=categories)

print("Weighted Normalized Data:")
display(weighted_data_df)

Weighted Normalized Data:


,Physical CPU Cores (PC),Total Memory (TM),Memory/Core (MC),Network Bandwidth (NB),On-Demand Hourly Cost (HC)
AWS hpc7g.4xlarge,0.005899,0.005689,0.015019,0.024480,0.009933
AWS hpc7g.8xlarge,0.011798,0.005689,0.007509,0.024480,0.009933
AWS hpc7g.16xlarge,0.023595,0.005689,0.003755,0.024480,0.009933
AWS hpc7a.12xlarge,0.008848,0.034135,0.060075,0.036719,0.042488
AWS hpc7a.24xlarge,0.017696,0.034135,0.030038,0.036719,0.042488
AWS hpc7a.48xlarge,0.035393,0.034135,0.015019,0.036719,0.042488
AWS hpc7a.96xlarge,0.070786,0.034135,0.007509,0.036719,0.042488
AWS hpc6id.32xlarge,0.023595,0.045513,0.030038,0.024480,0.033636
AWS hpc6a.48xlarge,0.035393,0.017067,0.007509,0.012240,0.016995
Azure Standard_HB60-15rs,0.005530,0.010134,0.028536,0.012240,0.013455


**Determination of Ideal Solution and Negative Ideal Solution**

Ideal Solution and Negative Ideal Solution are key concepts used to evaluate alternatives based on their distance from these ideal points.

In [40]:
# Determine the Ideal and Negative Ideal Solutions
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = weighted_data[:, j]
    max_val = np.max(column)
    min_val = np.min(column)

    if j in benefit_categories:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

ideal_df = pd.DataFrame(data=[a_pos, a_neg], index=["A+", "A-"], columns=categories)
print("Ideal and Negative Ideal Solutions:")
display(ideal_df)

Ideal and Negative Ideal Solutions:


,Physical CPU Cores (PC),Total Memory (TM),Memory/Core (MC),Network Bandwidth (NB),On-Demand Hourly Cost (HC)
A+,0.070786,0.062580,0.110144,0.048959,0.009933
A-,0.005530,0.005689,0.003755,0.012240,0.050986


**Calculation of Similarity Scores**

The core of TOPSIS lies in the calculation of similarity scores for each alternative with respect to the ideal and negative ideal solutions. The ideal solution represents the maximum (or minimum, depending on the nature of the criterion) values for each criterion, while the negative ideal solution represents the minimum (or maximum) values.

In [41]:
# Calculate the similarity scores
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = weighted_data[i] - a_pos
    diff_neg = weighted_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

similarity_scores_df = pd.DataFrame(data=zip(sp, sn, cs), index=alternatives, columns=["S+", "S-", "Ci"])
print("Similarity Scores:")
display(similarity_scores_df)

Similarity Scores:


,S+,S-,Ci
AWS hpc7g.4xlarge,0.130748,0.044296,0.253057
AWS hpc7g.8xlarge,0.133602,0.043457,0.245439
AWS hpc7g.16xlarge,0.131839,0.046492,0.260705
AWS hpc7a.12xlarge,0.091444,0.068291,0.427527
AWS hpc7a.24xlarge,0.106087,0.048160,0.312229
AWS hpc7a.48xlarge,0.110997,0.049993,0.310534
AWS hpc7a.96xlarge,0.112039,0.075849,0.403691
AWS hpc6id.32xlarge,0.100481,0.055262,0.354828
AWS hpc6a.48xlarge,0.123516,0.046805,0.274804
Azure Standard_HB60-15rs,0.122596,0.045193,0.269346


**Ranking of Alternatives**

The final step involves ranking the alternatives based on their relative closeness to the ideal solution and distance from the anti-ideal solution.

In [42]:
# Ranking of alternatives
initial_ranks = rankdata(-cs)
ranking_df = pd.DataFrame(data=zip(cs, initial_ranks), index=alternatives, columns=["TOPSIS Score", "Initial Rank"]).sort_values(by="Initial Rank")
print("Initial Ranking of Alternatives (Descending Order):")
display(ranking_df)

Initial Ranking of Alternatives (Descending Order):


,TOPSIS Score,Initial Rank
Azure Standard_HX176-24rs,0.629317,1.0
Azure Standard_HX176-48rs,0.497109,2.0
Azure Standard_HX176rs,0.466516,3.0
Azure Standard_HX176-144rs,0.451462,4.0
Azure Standard_HB176-24rs_v4,0.448079,5.0
Azure Standard_HX176-96rs,0.440334,6.0
AWS hpc7a.12xlarge,0.427527,7.0
Azure Standard_HB176rs_v4,0.406556,8.0
AWS hpc7a.96xlarge,0.403691,9.0
Azure Standard_HB120-16rs_v3,0.379673,10.5


**Sensitivity Analysis**

Sensitivity analysis in the context of TOPSIS is performed to evaluate the robustness of the rankings by examining how variations in the criteria weights affect the results. This analysis ensures that the final rankings are reliable and not overly sensitive to changes in the assigned weights.

In [77]:
# Sensitivity Analysis: Varying weights for each criterion
def sensitivity_analysis(raw_data, initial_weights, benefit_categories, alternatives):
    sensitivities = {}
    # Obtain initial ranking with current weights
    base_scores = topsis(raw_data, initial_weights, benefit_categories)
    base_ranking = rankdata(-base_scores)

    for i in range(len(initial_weights)):
        altered_weights = initial_weights.copy()
        for delta in np.linspace(-0.1, 0.1, 5):  # vary weights by ±10%
            if 0 <= initial_weights[i] + delta <= 1:
                altered_weights[i] = initial_weights[i] + delta
                # Ensure the weights sum to 1
                altered_weights /= np.sum(altered_weights)
                scores = topsis(raw_data, altered_weights, benefit_categories)
                ranking = rankdata(-scores)
                # Store the result using base_ranking as reference
                sensitivity_key = (i, delta)
                sensitivities[sensitivity_key] = pd.Series(ranking, index=alternatives)

    # Convert sensitivity results to DataFrame and align columns with initial ranking
    sensitivity_df = pd.DataFrame(sensitivities).T
    sensitivity_df.columns = alternatives  # Ensure correct column names for alternatives
    sensitivity_df.index.names = ['Criterion', 'Delta']
    sensitivity_df = sensitivity_df[ranking_df.sort_values("Initial Rank").index]

    return sensitivity_df

# Perform sensitivity analysis
sensitivity_df = sensitivity_analysis(raw_data, initial_weights, benefit_categories, alternatives)

print("Sensitivity Analysis:")
display(sensitivity_df)

Sensitivity Analysis:


Azure Standard_HX176-24rs  Azure Standard_HX176-48rs  \
Criterion Delta                                                         
0         -0.10                        1.0                        2.0   
          -0.05                        1.0                        2.0   
           0.00                        1.0                        2.0   
           0.05                        1.0                        3.0   
           0.10                        1.0                        6.0   
1         -0.10                        1.0                        2.0   
          -0.05                        1.0                        2.0   
           0.00                        1.0                        2.0   
           0.05                        1.0                        2.0   
           0.10                        1.0                        2.0   
2         -0.10                        3.0                        7.0   
          -0.05                        1.0                        4.0   
           0.00                        1.0                        2.0   
           0.05                        1.0                        2.0   
           0.10                        1.0                        2.0   
3         -0.10                        1.0                        2.0   
          -0.05                        1.0                        2.0   
           0.00                        1.0                        2.0   
           0.05                        1.0                        2.0   
           0.10                        1.0                        2.0   
4         -0.10                        1.0                        2.0   
          -0.05                        1.0                        2.0   
           0.00                        1.0                        2.0   
           0.05                        1.0                        2.0   
           0.10                        1.0                        2.0   

                 Azure Standard_HX176rs  Azure Standard_HX176-144rs  \
Criterion Delta                                                       
0         -0.10                     9.0                         8.0   
          -0.05                     6.0                         7.0   
           0.00                     3.0                         5.0   
           0.05                     2.0                         4.0   
           0.10                     2.0                         4.0   
1         -0.10                     4.0                         6.0   
          -0.05                     4.0                         6.0   
           0.00                     3.0                         5.0   
           0.05                     3.0                         4.0   
           0.10                     3.0                         4.0   
2         -0.10                     1.0                         2.0   
          -0.05                     2.0                         3.0   
           0.00                     3.0                         4.0   
           0.05                     5.0                         6.0   
           0.10                     7.0                         9.0   
3         -0.10                     3.0                         4.0   
          -0.05                     3.0                         4.0   
           0.00                     3.0                         4.0   
           0.05                     3.0                         4.0   
           0.10                     3.0                         5.0   
4         -0.10                     3.0                         4.0   
          -0.05                     3.0                         4.0   
           0.00                     3.0                         4.0   
           0.05                     3.0                         4.0   
           0.10                     3.0                         5.0   

                 Azure Standard_HB176-24rs_v4  Azure Standard_HX176-96rs  \
Criterion Delta                                    

**Bootstrapping Analysis**

Bootstrapping analysis is employed to evaluate the variability and stability of TOPSIS rankings by generating multiple resamples of the decision matrix and recalculating the TOPSIS scores for each resample. This approach helps in understanding the distribution of rankings and assessing the robustness of the decision outcomes.

In [82]:
# Bootstrapping Analysis: Generating bootstrap samples and calculating TOPSIS scores
def bootstrap_analysis(raw_data, initial_weights, benefit_categories, num_samples=100000):
    m, n = raw_data.shape
    bootstrap_scores = np.zeros((num_samples, m))

    for i in range(num_samples):
        bootstrap_sample_indices = np.random.choice(m, m, replace=True)
        bootstrap_sample = raw_data[bootstrap_sample_indices]
        bootstrap_scores[i] = topsis(bootstrap_sample, initial_weights, benefit_categories)

    return bootstrap_scores

bootstrap_scores = bootstrap_analysis(raw_data, initial_weights, benefit_categories)

# Analyzing the bootstrap results
bootstrap_ranks = np.array([rankdata(-scores) for scores in bootstrap_scores])
bootstrap_mean_ranks = np.mean(bootstrap_ranks, axis=0)
bootstrap_rank_intervals = np.percentile(bootstrap_ranks, [2.5, 97.5], axis=0)

# Display bootstrap analysis results
bootstrap_df = pd.DataFrame({
    "TOPSIS Score": topsis(raw_data, initial_weights, benefit_categories),
    "Initial Rank": initial_ranks,
    "Mean Rank": bootstrap_mean_ranks,
    "2.5% Rank": bootstrap_rank_intervals[0],
    "97.5% Rank": bootstrap_rank_intervals[1]
}, index=alternatives).sort_values(by="Initial Rank")

print("Bootstrap Analysis Results (Descending Order):")
display(bootstrap_df)

Bootstrap Analysis Results (Descending Order):


,TOPSIS Score,Initial Rank,Mean Rank,2.5% Rank,97.5% Rank
Azure Standard_HX176-24rs,0.629317,1.0,20.509140,1.5,39.5
Azure Standard_HX176-48rs,0.497109,2.0,20.528645,1.5,39.5
Azure Standard_HX176rs,0.466516,3.0,20.479365,1.5,39.5
Azure Standard_HX176-144rs,0.451462,4.0,20.510680,1.5,39.5
Azure Standard_HB176-24rs_v4,0.448079,5.0,20.502080,1.5,39.5
Azure Standard_HX176-96rs,0.440334,6.0,20.521395,1.5,39.5
AWS hpc7a.12xlarge,0.427527,7.0,20.469685,1.5,39.5
Azure Standard_HB176rs_v4,0.406556,8.0,20.493165,1.5,39.5
AWS hpc7a.96xlarge,0.403691,9.0,20.516070,1.5,39.5
Azure Standard_HB120-16rs_v3,0.379673,10.5,20.500435,1.5,39.5


**Non-Parametric Tests**

Non-parametric tests are utilized to evaluate the statistical significance of the differences in rankings obtained from the bootstrapping analysis. These tests do not assume a specific distribution for the data and are particularly useful for analyzing ordinal rankings.

In [83]:
# Non-parametric Tests: Friedman Test
def friedman_test(bootstrap_ranks):
    # Perform the Friedman test
    stat, p = friedmanchisquare(*bootstrap_ranks.T)
    return stat, p

# Perform the Friedman test
stat, p = friedman_test(bootstrap_ranks)
print(f"Friedman Test Statistic: {stat}, p-value: {p}")

# Adding Friedman Test p-value to summary table
bootstrap_df["Friedman Test p-value"] = p
print("Final Summary Table with Friedman Test p-value (Descending Order):")
display(bootstrap_df)

Friedman Test Statistic: 19.564334458561937, p-value: 0.9960176078038504
Final Summary Table with Friedman Test p-value (Descending Order):


,TOPSIS Score,Initial Rank,Mean Rank,2.5% Rank,97.5% Rank,Friedman Test p-value
Azure Standard_HX176-24rs,0.629317,1.0,20.509140,1.5,39.5,0.996018
Azure Standard_HX176-48rs,0.497109,2.0,20.528645,1.5,39.5,0.996018
Azure Standard_HX176rs,0.466516,3.0,20.479365,1.5,39.5,0.996018
Azure Standard_HX176-144rs,0.451462,4.0,20.510680,1.5,39.5,0.996018
Azure Standard_HB176-24rs_v4,0.448079,5.0,20.502080,1.5,39.5,0.996018
Azure Standard_HX176-96rs,0.440334,6.0,20.521395,1.5,39.5,0.996018
AWS hpc7a.12xlarge,0.427527,7.0,20.469685,1.5,39.5,0.996018
Azure Standard_HB176rs_v4,0.406556,8.0,20.493165,1.5,39.5,0.996018
AWS hpc7a.96xlarge,0.403691,9.0,20.516070,1.5,39.5,0.996018
Azure Standard_HB120-16rs_v3,0.379673,10.5,20.500435,1.5,39.5,0.996018
